In [90]:
import xgboost as xgb
import pandas as pd
import numpy as np

In [91]:
train = pd.read_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/train.csv')

In [92]:
train.head(5)

,Unnamed: 0,itemID,categoryID,title,description,images_array,attrsJSON,price,locationID,metroID,lat,lon,isDuplicate,generationMethod,regionID,parentCategoryID
0,0,1,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"1064094, 5252822, 6645873, 6960145, 9230265","{""Вид техники"":""Грузовики""}",300000.0,648140,NaN,64.686946,30.815924,1,1,648070,1
1,1,3,14,Yamaha r6,Весь в тюнинге.,"11919573, 14412228, 3204180, 6646877","{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""С...",300000.0,639040,NaN,55.678037,37.256548,1,1,637680,1
2,2,4,84,iPhone 3gs 8gb,"Телефон в хорошем состоянии, трещин и сколов н...","14384831, 6102021","{""Вид телефона"":""iPhone""}",3500.0,640650,NaN,56.239398,43.460458,0,1,640310,6
3,3,7,84,Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской ap...",NaN,"{""Вид телефона"":""Другие марки""}",13500.0,662210,NaN,55.777170,37.586194,1,1,662000,6
4,4,8,39,Лыжные ботинки,"Лыжные ботинки в хорошем состоянии, 34 размер","13718854, 4787310","{""Вид товара"":""Зимние виды спорта""}",500.0,624360,NaN,55.777170,37.586194,1,1,624300,7


In [93]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982792 entries, 0 to 5982791
Data columns (total 16 columns):
Unnamed: 0          int64
itemID              int64
categoryID          int64
title               object
description         object
images_array        object
attrsJSON           object
price               float64
locationID          int64
metroID             float64
lat                 float64
lon                 float64
isDuplicate         int64
generationMethod    int64
regionID            int64
parentCategoryID    int64
dtypes: float64(4), int64(8), object(4)
memory usage: 730.3+ MB


Classification Models: LDA, QDA, Logistic regression, SVM, Decision Trees, Random Forests, various forms of Boosting

# 1 Tree based methods

## 1.1 Decision Tree

In [94]:
#metroID有太多 missing data
train1 = train.drop(['Unnamed: 0', 'title', 'description', 'images_array', 'attrsJSON', 'generationMethod', 'metroID'], axis=1)

In [95]:
train2 = train1.dropna()

In [96]:
train2.head()

,itemID,categoryID,price,locationID,lat,lon,isDuplicate,regionID,parentCategoryID
0,1,81,300000.0,648140,64.686946,30.815924,1,648070,1
1,3,14,300000.0,639040,55.678037,37.256548,1,637680,1
2,4,84,3500.0,640650,56.239398,43.460458,0,640310,6
3,7,84,13500.0,662210,55.777170,37.586194,1,662000,6
4,8,39,500.0,624360,55.777170,37.586194,1,624300,7


In [97]:
y = train2[['isDuplicate']]

In [98]:
X = train2.drop('isDuplicate',axis=1)

In [99]:
from sklearn.model_selection import train_test_split

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [101]:
from sklearn.tree import DecisionTreeClassifier

In [102]:
dtree = DecisionTreeClassifier()

In [103]:
dtree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [104]:
prediction = dtree.predict(X_test)

In [106]:
prediction

array([0, 1, 0, ..., 1, 0, 0])

In [107]:
from sklearn.metrics import classification_report, confusion_matrix

In [108]:
print(confusion_matrix(y_test, prediction))

[[898187 151292]
 [220086 517276]]


In [109]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.80      0.86      0.83   1049479
           1       0.77      0.70      0.74    737362

   micro avg       0.79      0.79      0.79   1786841
   macro avg       0.79      0.78      0.78   1786841
weighted avg       0.79      0.79      0.79   1786841



We have a pretty good prediction already for the decision tree with high precision.


## 1.2 Random Forest

In [110]:
from sklearn.ensemble import RandomForestClassifier

In [111]:
rfc = RandomForestClassifier(n_estimators = 30)

In [112]:
rfc.fit(X_train, y_train)

/Users/jerry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [113]:
rfc_pred = rfc.predict(X_test)

In [114]:
print(confusion_matrix(y_test, rfc_pred))

[[899771 149708]
 [189189 548173]]


In [115]:
print(classification_report(y_test, rfc_pred))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84   1049479
           1       0.79      0.74      0.76    737362

   micro avg       0.81      0.81      0.81   1786841
   macro avg       0.81      0.80      0.80   1786841
weighted avg       0.81      0.81      0.81   1786841



There is no huge improvement over the simple decision tree method. 